<h1><center> Спортивные школы </center></h1>

<h2>Описание проекта</h2>

Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.


<h2>Цель:</h2>

Создать решение для стандартизации названий спортивных школ.
Одна и та же школа может быть записана по разному, нужно сопоставить эти варианты эталонному названию из предоставленной таблицы.

<h3> Исходные данные: </h3>

Файлы:
* школы.csv
* примерное написание.csv

## Предварительный анализ данных

In [190]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import (
SentenceTransformer,
InputExample,
util, losses,
)
from torch.utils.data import DataLoader

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [ ]:
pip install nlpaug

In [148]:
school = pd.read_csv('школы.csv')

In [149]:
school

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург
...,...,...,...
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край


In [185]:
appr_school = pd.read_csv('примерное написание.csv')

In [186]:
appr_school

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""
...,...,...
890,3,"Республика Татарстан, СШОР ФСО Авиатор"
891,3,"СШОР ФСО Авиатор, Республика Татарстан"
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
893,2,"ЯНАО, СШ ""Авангард"""


## Предобработка данных

In [152]:
school['title'] = school['name'].replace(r'\["«»/-]', ' ', regex=True).replace(r'[^А-Яа-яёЁA-Za-z\s]', ' ', regex=True).replace(r'\s+', ' ', regex=True).str.strip()

In [153]:
school['title'] = school['region'].replace(r'\["«»/-]', ' ', regex=True).replace(r'[^А-Яа-яёЁA-Za-z\s]', ' ', regex=True).replace(r'\s+', ' ', regex=True).str.strip() + ' ' + school['title']

In [154]:
school

,school_id,name,region,title
0,1,Авангард,Московская область,Московская область Авангард
1,2,Авангард,Ямало-Ненецкий АО,Ямало Ненецкий АО Авангард
2,3,Авиатор,Республика Татарстан,Республика Татарстан Авиатор
3,4,Аврора,Санкт-Петербург,Санкт Петербург Аврора
4,5,Ice Dream / Айс Дрим,Санкт-Петербург,Санкт Петербург Ice Dream Айс Дрим
...,...,...,...,...
301,305,Прогресс,Алтайский край,Алтайский край Прогресс
302,609,"""СШ ""Гвоздика""",Удмуртская республика,Удмуртская республика СШ Гвоздика
303,610,"СШОР ""Надежда Губернии",Саратовская область,Саратовская область СШОР Надежда Губернии
304,611,КФК «Айсберг»,Пермский край,Пермский край КФК Айсберг


### С помощь аугментаций увеличиваем количество данных:

In [155]:
aug = naw.RandomWordAug(action='swap')
school['title_aug'] = school.title.transform(lambda x: aug.augment(x, 4))
aug = aug = nac.RandomCharAug(action="delete")
school['title_aug'] = school.title.transform(lambda x: aug.augment(x, 16))

In [156]:
school

,school_id,name,region,title,title_aug
0,1,Авангард,Московская область,Московская область Авангард,"[Московская область Аанар, Московская область ..."
1,2,Авангард,Ямало-Ненецкий АО,Ямало Ненецкий АО Авангард,"[Ямало неций АО Авнрд, Ямало Неней АО Аанар, Я..."
2,3,Авиатор,Республика Татарстан,Республика Татарстан Авиатор,"[Республика Татарстан виат, Республика аарстн ..."
3,4,Аврора,Санкт-Петербург,Санкт Петербург Аврора,"[Санкт етрбуг Аврора, Санкт Петербург Аора, Са..."
4,5,Ice Dream / Айс Дрим,Санкт-Петербург,Санкт Петербург Ice Dream Айс Дрим,"[нкт Петербург Ice Dream Айс Дм, акт Петербург..."
...,...,...,...,...,...
301,305,Прогресс,Алтайский край,Алтайский край Прогресс,"[Алтайский ай Прогресс, тайсий край Прогресс, ..."
302,609,"""СШ ""Гвоздика""",Удмуртская республика,Удмуртская республика СШ Гвоздика,"[мутская республика СШ Гзика, дмуртая респбла ..."
303,610,"СШОР ""Надежда Губернии",Саратовская область,Саратовская область СШОР Надежда Губернии,"[Саратовская область ОР ежда Губернии, Саратов..."
304,611,КФК «Айсберг»,Пермский край,Пермский край КФК Айсберг,"[Прмси край КФК Айср, мский рй КФК Айсберг, Пс..."


In [316]:
df = school.explode('title_aug')
df

,school_id,name,region,title,title_aug
0,1,Авангард,Московская область,Московская область Авангард,Московская область Аанар
0,1,Авангард,Московская область,Московская область Авангард,Московская область Аанар
0,1,Авангард,Московская область,Московская область Авангард,Московская олас Авангард
0,1,Авангард,Московская область,Московская область Авангард,Моковса область Авангард
0,1,Авангард,Московская область,Московская область Авангард,Московская олаь Авангард
...,...,...,...,...,...
305,1836,"ООО ""Триумф""",Москва,Москва ООО Триумф,Москва ООО иумф
305,1836,"ООО ""Триумф""",Москва,Москва ООО Триумф,Москва ООО румф
305,1836,"ООО ""Триумф""",Москва,Москва ООО Триумф,Мока ООО Триумф
305,1836,"ООО ""Триумф""",Москва,Москва ООО Триумф,Мова ООО Триумф


In [318]:
df = df.drop_duplicates().reset_index(drop=True)
df = df[['title', 'title_aug']]
df

,title,title_aug
0,Московская область Авангард,Московская область Аанар
1,Московская область Авангард,Московская олас Авангард
2,Московская область Авангард,Моковса область Авангард
3,Московская область Авангард,Московская олаь Авангард
4,Московская область Авангард,Московская обат Авангард
...,...,...
4536,Москва ООО Триумф,Мска ООО Триумф
4537,Москва ООО Триумф,оква ООО Триумф
4538,Москва ООО Триумф,Москва ООО иумф
4539,Москва ООО Триумф,Москва ООО румф


## Обучение моделей

In [319]:
train, test = train_test_split(df, test_size=0.25, random_state=12345)

In [283]:
model = SentenceTransformer('sentence-transformers/LaBSE')

In [320]:
corpus = model.encode(df.title.values)

In [321]:
query = model.encode(test.title_aug.values)

In [322]:
search_result = util.semantic_search(query, corpus, top_k=1)

In [323]:
test['candidate_idx'] = [x[0]['corpus_id'] for x in search_result]

In [324]:
test['candidate_name'] = df.title.values[test.candidate_idx.values]

In [325]:
test = test.reset_index(drop=True)
test

,title,title_aug,candidate_idx,candidate_name
0,Санкт Петербург СК ФК Open Ice,Санкт Птебуг СК ФК pe Ice,4187,Санкт Петербург СК ФК Ice Prestige
1,Республика Коми МУ СШ,еспблиа ом МУ СШ,3127,Москва СШОР
2,Белгородская область ДЮСШ по ЗВС,Белгородская оать ДЮ по ЗВС,752,Белгородская область ДЮСШ по ЗВС
3,Саратовская область СШОР Надежда Губернии,аатосая область СШОР Надежда Губрн,4496,Саратовская область СШОР Надежда Губернии
4,Санкт Петербург Чемпион,Санкт Петербург пион,1980,Санкт Петербург Прибой
...,...,...,...,...
1131,Республика Саха РССШ по ЗВС,епублиа Саха СС по ЗВС,2111,Республика Саха РССШ по ЗВС
1132,Нижегородская область СШОР по ЛВС,Нжородскя область СР по ЛВС,2869,Нижегородская область СШОР по ЛВС
1133,Свердловская область СШ,Свероскя область СШ,2735,Свердловская область СШ
1134,Санкт Петербург Прибой,Санкт Петербург Прий,1980,Санкт Петербург Прибой


In [326]:
# процент правильных ответов
((test.title == test.candidate_name).sum())/ len(test)

0.8820422535211268

### Дообучение модели

In [327]:
make_example = lambda x: InputExample(texts=[x['title'], x['title_aug']])

In [328]:
examples = train[['title', 'title_aug']].apply(make_example, axis=1).values

In [329]:
train_dataloader = DataLoader(examples, shuffle=True, batch_size=16)

In [330]:
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [331]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/213 [00:00<?, ?it/s]

Iteration:   0%|          | 0/213 [00:00<?, ?it/s]

## Тестирование модели

In [332]:
df = school[['school_id', 'title', 'title_aug']]

In [336]:
corpus = model.encode(df.title.values)
query = model.encode(appr_school.name.values)
search_result = util.semantic_search(query, corpus, top_k=1)
appr_school['candidate_idx'] = [x[0]['corpus_id'] for x in search_result]
appr_school['candidate_name'] = df.title.values[appr_school.candidate_idx.values]

In [337]:
appr_school

,school_id,name,candidate_idx,candidate_name
0,1836,"ООО ""Триумф""",305,Москва ООО Триумф
1,1836,"Москва, СК ""Триумф""",305,Москва ООО Триумф
2,610,"СШОР ""Надежда Губернии",303,Саратовская область СШОР Надежда Губернии
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",303,Саратовская область СШОР Надежда Губернии
4,609,"""СШ ""Гвоздика""",302,Удмуртская республика СШ Гвоздика
...,...,...,...,...
890,3,"Республика Татарстан, СШОР ФСО Авиатор",142,Республика Татарстан РСШОР
891,3,"СШОР ФСО Авиатор, Республика Татарстан",2,Республика Татарстан Авиатор
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",229,Республика Татарстан ФАУ МО РФ ЦСКА
893,2,"ЯНАО, СШ ""Авангард""",1,Ямало Ненецкий АО Авангард


# Выводы:

С помощью аугментаций я увеличила количество данных, и дальше, используя семантический поиск, я сопоставила варианты написания школ эталонному названию из предоставленной таблицы. Результаты для каждого названия я предоставила в таблице выше.